In [1]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load in 

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the "../input/" directory.
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # Any results you write to the current directory are saved as output.

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df=pd.read_csv('WA_Fn-UseC_-Telco-Customer-Churn.csv')

FileNotFoundError: File b'/kaggle/input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv' does not exist

In [ ]:
pd.options.display.max_columns=None

In [ ]:
df.shape

In [ ]:
df.drop('customerID',axis=1,inplace=True)

In [ ]:
df.select_dtypes(include='object').head(2)

In [ ]:
# MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,
# StreamingTV,StreamingMovies,Contract

In [ ]:
df['gender']=df['gender'].replace({'Male':1,'Female':0})
df['Partner']=df['Partner'].replace({'Yes':1,'No':0})
df['Dependents']=df['Dependents'].replace({'Yes':1,'No':0})
df['PhoneService']=df['PhoneService'].replace({'Yes':1,'No':0})
df['PaperlessBilling']=df['PaperlessBilling'].replace({'Yes':1,'No':0})
df['Churn']=df['Churn'].replace({'Yes':1,'No':0})

In [ ]:
# Notes:
#   1-We have to convert all categorical variables by using manual label encoding.
#   2-fix the datatype of TotalCharges which is numerical feature but it appears to be object datatype.
#   3- Replace null values by respective mean/median.

# for i in df.select_dtypes(include='object'):
#     if df[i].nunique()>2:
#         print(i,':\n',df[i].value_counts())

In [ ]:
df['TotalCharges']=pd.to_numeric(df['TotalCharges'],errors='coerce')

In [ ]:
df['TotalCharges']=df['TotalCharges'].fillna(df['TotalCharges'].median())

In [ ]:
dfcopy=pd.get_dummies(df.select_dtypes(include='object'),drop_first=True)

In [ ]:
df=pd.concat([df,dfcopy],axis=1)

In [ ]:
df.select_dtypes(include='object').columns

In [ ]:
df.drop(['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
       'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies',
       'Contract', 'PaymentMethod'],axis=1,inplace=True)

In [ ]:
# df is clean dataframe

## Importing Libraries:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
import seaborn as sns

# from sklearn.preprocessing import LabelEncoder,OneHotEncoder #this is optional

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import scale
from scipy.stats import zscore

from sklearn.model_selection import train_test_split,KFold,StratifiedKFold,GridSearchCV,cross_val_score
from sklearn.ensemble import RandomForestClassifier,BaggingClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.linear_model import LinearRegression,LogisticRegression,Lasso, Ridge
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier


from sklearn.metrics import r2_score,roc_auc_score,classification_report,mean_squared_error,accuracy_score,confusion_matrix


import warnings
warnings.filterwarnings('ignore')

# 1-Logistic and Naive Bayes Classifier:

In [ ]:
X=df.drop('Churn',axis=1)
y=df['Churn']

In [ ]:
lr = LogisticRegression()
gb = GaussianNB()
models = []
models.append(('LogisticRegression',lr))
models.append(('NaviveBayes',gb))

In [ ]:
results = []
names = []
for name, model in models:
    kfold = KFold(shuffle=True, n_splits=5, random_state=0)
    cv_results = cross_val_score(model, X, y, cv=kfold, scoring='roc_auc')
    results.append(1 - cv_results)
    names.append(name)
    print('%s : %f(%f)' %(name,np.mean(cv_results), np.var(cv_results,ddof=1)))
# boxplot algorithm comparision
fig = plt.figure()
fig.suptitle('Algorithm Comparision')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()

In [ ]:
#Splitting X&y using train_test:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=.2,random_state=8)

In [ ]:
logreg=LogisticRegression()
logreg.fit(X_train,y_train)
y_prob_train = logreg.predict_proba(X_train)
y_pred_train = logreg.predict(X_train)
y_prob_test = logreg.predict_proba(X_test)
y_pred_test = logreg.predict(X_test)

print('Confusion Matrix - Train:', '\n', confusion_matrix(y_train, y_pred_train))
print('Overall Accuracy', accuracy_score(y_train, y_pred_train))

print('Confusion Matrix - Test:', '\n', confusion_matrix(y_test, y_pred_test))
print('Overall Accuracy', accuracy_score(y_test, y_pred_test))
      
from sklearn.metrics import log_loss
print('log loss: ',log_loss(y_test,y_prob_test))

# 2 - KNN Classifier

In [ ]:
X=df.drop('Churn',axis=1)
y=df['Churn']
sc = StandardScaler()
X_scaled = sc.fit_transform(X)

In [ ]:
knn = KNeighborsClassifier()
knn_params = {'n_neighbors':np.arange(3,20), 'weights':['uniform','distance']}
gscv = GridSearchCV(knn, knn_params, cv=5, scoring='roc_auc')
gscv.fit(X_scaled, y)
print(gscv.best_params_)

In [ ]:
gscv_best_knn=gscv.best_params_

KNN=KNeighborsClassifier(**gscv_best_knn)

KNN.fit(X_scaled,y)
KNN.score(X_scaled,y)

In [ ]:
#Splitting X&y using train_test:
X_train,X_test,y_train,y_test=train_test_split(X_scaled,y,test_size=.2,random_state=8)

In [ ]:
y_prob_train = KNN.predict_proba(X_train)
y_pred_train = KNN.predict(X_train)
y_prob_test = KNN.predict_proba(X_test)
y_pred_test = KNN.predict(X_test)

print('Confusion Matrix - Train:', '\n', confusion_matrix(y_train, y_pred_train))
print('Overall Accuracy', accuracy_score(y_train, y_pred_train))

print('Confusion Matrix - Test:', '\n', confusion_matrix(y_test, y_pred_test))
print('Overall Accuracy', accuracy_score(y_test, y_pred_test))
      
from sklearn.metrics import log_loss
print('log loss: ',log_loss(y_test,y_prob_test))

# 3- Decision tree: [No need to do scaling now]

In [ ]:
X=df.drop('Churn',axis=1)
y=df['Churn']

In [ ]:
dt = DecisionTreeClassifier()
dt_params = {'max_depth':np.arange(1,10), 'min_samples_leaf':np.arange(2,100), 'criterion':['entropy','gini']}
gscv = GridSearchCV(dt, dt_params, cv=5, scoring='roc_auc')
gscv.fit(X, y)
print(gscv.best_params_)

In [ ]:
gscv_best_DT=gscv.best_params_
DT=DecisionTreeClassifier(**gscv_best_DT)
DT.fit(X,y)
DT.score(X,y)

In [ ]:
#Splitting X&y using train_test:
X_train,X_test,y_train,y_test=train_test_split(X_scaled,y,test_size=.2,random_state=8)


DT.fit(X_train,y_train)

In [ ]:
y_prob_train = DT.predict_proba(X_train)
y_pred_train = DT.predict(X_train)
y_prob_test = DT.predict_proba(X_test)
y_pred_test = DT.predict(X_test)

print('Confusion Matrix - Train:', '\n', confusion_matrix(y_train, y_pred_train))
print('Overall Accuracy', accuracy_score(y_train, y_pred_train))

print('Confusion Matrix - Test:', '\n', confusion_matrix(y_test, y_pred_test))
print('Overall Accuracy', accuracy_score(y_test, y_pred_test))
      
from sklearn.metrics import log_loss
print('log loss: ',log_loss(y_test,y_prob_test))

# 4 - Random Forest

In [ ]:
X=df.drop('Churn',axis=1)
y=df['Churn']

In [ ]:
auc_avg = []
auc_var = []
for ne in np.arange(1,100):
    RF=RandomForestClassifier(n_estimators=ne,random_state=0)
    kfold = KFold(shuffle=True,n_splits=5,random_state=0)
    auc = cross_val_score(RF, X, y, cv=kfold, scoring='roc_auc')
    auc_avg.append(1 - np.mean(auc))
    auc_var.append(np.var(auc,ddof=1))

In [ ]:
print('Min Bias Error:',np.min(auc_avg),' n_estimator:',np.argmin(auc_avg)+1,' Variance Error:',auc_var[np.argmin(auc_avg)])
print('Bias Error:',auc_avg[np.argmin(auc_var)],' n_estimator:',np.argmin(auc_var)+1,'Min Variance Error:',np.min(auc_var))

In [ ]:
#Splitting X&y using train_test:
X_train,X_test,y_train,y_test=train_test_split(X_scaled,y,test_size=.2,random_state=8)

In [ ]:
RF=RandomForestClassifier(n_estimators=16)
RF.fit(X,y)
RF.score(X,y)

In [ ]:
RF.fit(X_train,y_train)

In [ ]:
y_prob_train = RF.predict_proba(X_train)
y_pred_train = RF.predict(X_train)
y_prob_test = RF.predict_proba(X_test)
y_pred_test = RF.predict(X_test)

print('Confusion Matrix - Train:', '\n', confusion_matrix(y_train, y_pred_train))
print('Overall Accuracy', accuracy_score(y_train, y_pred_train))

print('Confusion Matrix - Test:', '\n', confusion_matrix(y_test, y_pred_test))
print('Overall Accuracy', accuracy_score(y_test, y_pred_test))
      
from sklearn.metrics import log_loss
print('log loss: ',log_loss(y_test,y_prob_test))

# Comparison so far..!!

In [ ]:
X=df.drop('Churn',axis=1)
y=df['Churn']

In [ ]:
lr = LogisticRegression()
gb = GaussianNB()
knn=KNeighborsClassifier(**gscv_best_knn)
dt = DecisionTreeClassifier(**gscv_best_DT)
rf=RandomForestClassifier(n_estimators=16,random_state=0)
models = []
models.append(('LogisticRegression',lr))
models.append(('NaiveBayes',gb))
models.append(('KNeighborsClassifier',knn))
models.append(('DecisionTreeClassifier',dt))
models.append(('RandomForestClassifier',rf))

In [ ]:
results = []
names = []
for name, model in models:
    kfold = KFold(shuffle=True, n_splits=5, random_state=0)
    cv_results = cross_val_score(model, X_scaled, y, cv=kfold, scoring='roc_auc')
    results.append(1-cv_results)
    names.append(name)
    print('%s : %f(%f)' %(name,1 - np.mean(cv_results), np.var(cv_results,ddof=1)))
# boxplot algorithm comparision
fig = plt.figure(figsize=(15,5))
fig.suptitle('Algorithm Comparision')
ax = fig.add_subplot(111)
plt.boxplot(results)
ax.set_xticklabels(names)
plt.show()